# **Part 2**

# **Dataset Preparation & Preprocessing**

In [18]:
# đọc dữ liệu
import pandas as pd
data = pd.read_csv('resources/iris.data', header = None)
value = data.select_dtypes(include=["float64", "int64"])
label = data.select_dtypes(exclude=["float64", "int64"])
value

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [19]:
# Xử lý nhãn dư, đếm nhãn
import numpy as np
label_list = pd.unique(label[label.columns[0]])
label_list = label_list.tolist()
num_class = len(label_list)
label_data =label[label.columns[0]].values.tolist()
label_count = np.array(label[label.columns[0]].value_counts())

In [20]:
import numpy as np
X = np.array(value)
n = X.shape[0]
p = X.shape[1]
print(X.shape)
X

(150, 4)


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

# **Funtion**

In [21]:
c=0
def setC(i = 3):
    global c
    c = i

In [22]:
import random
def generate_M(m, m1, k, c=3):
    global M
    global index_x_giamsat
    index_x_giamsat = random.sample(range(n),k)# CHọn k1 vector x từ n vector x ban đầu  
    index_x_giamsat = np.sort(index_x_giamsat)
    M = np.zeros((n,c))
    for i in range(n):
        for j in range(c):
            M[i,j] = m
        if i in index_x_giamsat:
            c1= random.randint(0,c-1)
            M[i,c1] = m1

In [23]:

def generate_V(c):
    global V, V_truoc
    V = np.random.rand(c,p) 
    V_truoc = V

In [24]:
def generate_U(c):
    global U
    U = np.zeros((n,c))


In [ ]:
import math

def update_D(n,c):
    global D 
    D = np.zeros((n,c))   
    
    for i in range (n):
        for k in range(c):
            D[i][k] = math.sqrt(
                                sum(  pow(X[i] - V[k], 2))
                                )

In [ ]:
def solve_mu(sum_mu_i, d_ik, m, m1, epsilon):
    mu = 0
    vp = pow(1/(m1*d_ik*d_ik),1/(m1-1))
    vt = -1
    left = 0.0
    right = 1.0
    while (abs(vt-vp) > epsilon):
          mu = (right + left)/2
          vt = mu/pow(mu + sum_mu_i, (m1-m)/(m1-1))  
          if vt<vp:
                left = mu
          else:
                right =mu
    return mu

In [ ]:

def update_U(m,m1,c,epsilon):
    
    global U
    update_D(n,c)
    for i in range(n):
        ##Cong thuc 15
        if i not in index_x_giamsat:
            for k in range(c):
                mau_so = sum(pow( D[i][k] / D[i], 2/(m-1)))
                U[i][k] = 1/mau_so
        ##Cong thuc 16
        elif i in index_x_giamsat :
            d_min = np.amin(D[i])
            d_i = D[i]/d_min   #Cong thuc 17
            mu_i = np.zeros(c)
            #Cong thuc 18
            for j in range(c) :
                if (M[i][j] == m):
                    mu_i[j] = 1/pow( m *d_i[j]*d_i[j] , 1/(m-1) )
            sum_mu_i = sum(mu_i)
            #Cong thuc 19
            for j in range(c) :
                if (M[i][j] == m1):
                    mu_i[j] = solve_mu(sum_mu_i, d_i[j], m, m1, epsilon)
            for j in range(c) : 
                U[i][j] = mu_i[j]/sum(mu_i)
                   
            
                    

In [11]:
def update_V(c):
    V_temp = np.zeros((c,p))
    for k in range(c):
        temp = pow((U.T)[k],(M.T)[k])
        tu_so = np.zeros(p)
        for i in range(n):
            tu_so += temp[i] * X[i]
        V_temp[k] =  tu_so / sum(temp)
    return V_temp

In [12]:
def thuat_toan(m,m1,c,k,epsilon):
    setC(c)
    generate_V(c)
    generate_U(c)
    generate_M(m,m1,k,c)
    global V, V_truoc
    Epsilon = np.zeros((c,p)) + epsilon
    while True:
        update_U(m,m1,c,epsilon)
        V_truoc =V
        V= update_V(c)
        delta_V = abs(V- V_truoc)
        ktra = np.less_equal(delta_V, Epsilon)
        if (np.all(ktra)):
            break
    print("====================================================================================")
    print("=======================================Kết quả======================================")
    print("====================================================================================")
    print("Tâm cụm :")
    print(V)
    print("\n\n")
    

# **Evaluate**

In [13]:
# đếm số lượng phần tử mỗi cụm tương ứng với từng lớp nhãn dữ liệu
def count_class(num_class, num_cluster):
    global count_class_cluster
    count_class_cluster = np.zeros((num_class,num_cluster), dtype="int64")
    for k in range(n):
        k_class = label_list.index(label_data[k])
        index_max = np.argmax(U[k])
        count_class_cluster[k_class][index_max] +=1   
    # Ma trận có num_class dòng và num_cluster cột, phần tử thứ i,j của ma trận thuộc class i, cluster j

In [14]:
# Các độ đo
def external_validity(num_class, c):
    a1 = b1 = c1 = d1= 0
    for i in range(num_class):
        a1+= sum ( (count_class_cluster[i]-1)*count_class_cluster[i]/2 )
        b1+= (pow(sum(count_class_cluster[i]),2) - sum(pow(count_class_cluster[i],2)) )/2
    for i in range(num_class-1):
        for j in range(i+1, num_class):
            for k in range(c):
                c1+=count_class_cluster[i][k] * count_class_cluster[j][k]
                for h in range(c):
                    if h!=k :
                        d1+=count_class_cluster[i][k] * count_class_cluster[j][h]  
    print("====================================================================================")
    print("=======================================Đánh giá=====================================")
    print("====================================================================================")
    print()
    print(count_class_cluster)
    print()
    print("a1= "+str(a1))
    print("b1= "+str(b1))
    print("c1= "+str(c1))
    print("d1= "+str(d1))
    print()
    # Rand int
    w1 =  (a1 + d1)/ (a1 +b1 + c1 +d1)
    # Adjusted Rand Index
    M= a1 + b1 +c1 +d1
    w2 = (a1 - (a1+c1)*(a1+b1)/M) /( (2*a1+b1+c1)/2 - (a1+c1)*(a1+b1)/M )
    #Jaccard Coefficient
    w3 = a1 / (a1+b1+c1)
    print ("Rand Index          :" +str(w1))
    print ("Adjusted Rand Index :" + str(w2))
    print ("Jaccard Coefficient :" +str(w3))
    

In [15]:
def Meam_accuraccy():
    sum_cluster = sum(count_class_cluster)
    mean_accuraccy = sum_cluster/label_count
    print("Mean accuraccy      :"+ str(np.amin(mean_accuraccy)))
        

In [16]:
def evaluate(num_class, numcluster):
    count_class(num_class, numcluster)
    external_validity(num_class, numcluster)

# **Main**

In [17]:
thuat_toan(2,3,3,50,1e-6)
evaluate(3,3)

=======================================Kết quả======================================
Tâm cụm :
[[6.7781554  3.05064351 5.657371   2.05816341]
 [4.99983579 3.39694943 1.48434716 0.25210128]
 [5.90987991 2.7746695  4.37816383 1.40421084]]



=======================================Đánh giá=====================================

[[ 0 50  0]
 [ 3  0 47]
 [37  0 13]]

a1= 3053.0
b1= 622.0
c1= 722
d1= 6778

Rand Index          :0.8797315436241611
Adjusted Rand Index :0.7294203486015404
Jaccard Coefficient :0.694337047987264
